In [1]:
import requests
import pandas as pd
import datetime
from datetime import datetime, timezone

def to_epoch(my_date):
    my_date_dt = datetime.strptime(my_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
    epoch_timestamp = my_date_dt.timestamp()
    return int(epoch_timestamp)

def tick_to_price(tick, decimals0, decimals1):
    return 1.0001 ** - tick * (10 ** (decimals1 - decimals0))

def fetch_uniswap_swaps(pool_address, start_date, end_date, limit=1000):
    endpoint = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
    query = """
    query MyQuery($pool: String!, $first: Int!, $end_date: Int!, $skip: Int!) {
      swaps(
        where: {pool: $pool, transaction_: {timestamp_lte: $end_date}}
        orderBy: timestamp
        orderDirection: desc
        first: $first
        skip: $skip
      ) {
        amount0
        amount1
        amountUSD
        origin
        tick
        sqrtPriceX96
        sender
        recipient
        timestamp
        recipient
        id
        sqrtPriceX96
        tick
        token1 {
          decimals
          symbol
        }
        token0 {
          decimals
          symbol
        }
        transaction {
          gasUsed
          gasPrice
          blockNumber
          timestamp
        }
      }
    }
    """

    swaps = []
    skip = 0

    variables = {"pool": pool_address, "end_date": end_date, "first": limit, "skip": skip}
    response = requests.post(endpoint, json={'query': query, 'variables': variables})
    try:
        data = response.json()['data']['swaps']
    except:
        pass
        
    df = pd.DataFrame(data)
    print(datetime.utcfromtimestamp(int(df.timestamp.iloc[-1])))
    
    df[['amount0', 'amount1', 'amountUSD', 'timestamp']] = df[['amount0', 'amount1', 'amountUSD', 'timestamp']].astype('float')
    df[['sender', 'recipient', 'origin']] = df[['sender', 'recipient', 'origin']].astype('str')

    df['gasUsed']  = df['transaction'].apply(lambda x: x['gasUsed']).astype('int')
    df['gasPrice_wei'] = df['transaction'].apply(lambda x: float(x['gasPrice']))
    df['gasPrice_eth'] = df['gasPrice_wei'] * 10**-18
    
    df['blockNumber'] = df['transaction'].apply(lambda x: x['blockNumber']).astype('int')
    #df['timestamp'] = df['transaction'].apply(lambda x: x['timestamp']).astype('int')
    
    df['symbol0'] = df['token0'].apply(lambda x: x['symbol']).astype('str')
    df['symbol1'] = df['token1'].apply(lambda x: x['symbol']).astype('str')

    df['decimal0'] = df['token0'].apply(lambda x: x['decimals']).astype('int')
    df['decimal1'] = df['token1'].apply(lambda x: x['decimals']).astype('int')

    df['tick']  = df['tick'].astype('int')
    df['price'] = df.apply(lambda row: tick_to_price(row['tick'], row['decimal0'], row['decimal1']), axis=1)

    df['tcost_usd'] = df['gasPrice_eth']*df['price']*df['gasUsed']
    
    df['time'] = pd.to_datetime(df['timestamp'], unit='s')

    df = df.drop(['token1', 'token0', 'tick', 'decimal0', 'decimal1', 'id', 'sqrtPriceX96', 'transaction'], axis=1)

    return df

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
pool_address = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
pool_address = pool_address.lower()

start_date = "2024-01-01 00:00:00"
start_date = to_epoch(start_date)
end_date = "2024-01-10 23:59:59"
end_date = to_epoch(end_date)


In [3]:
# Getting the data from TheGraph and generating the df dataframe
df = fetch_uniswap_swaps(pool_address, start_date, end_date)
while df['timestamp'].iloc[-1]>start_date:
    df = pd.concat([df, fetch_uniswap_swaps(pool_address, start_date, int(df['timestamp'].iloc[-1]-1))], ignore_index=True)
df

df.to_pickle("data_UV3_WETH_USDC_January2024.pkl") #Saving the data locally

2024-01-10 22:05:23
2024-01-10 19:43:59
2024-01-10 16:40:23
2024-01-10 13:21:47
2024-01-10 09:51:35
2024-01-10 05:44:35
2024-01-10 00:59:47
2024-01-09 22:27:59
2024-01-09 20:42:35
2024-01-09 16:41:11
2024-01-09 12:59:23
2024-01-09 08:45:35
2024-01-09 04:00:35
2024-01-09 00:07:23
2024-01-08 20:21:59
2024-01-08 17:50:35
2024-01-08 14:11:59
2024-01-08 10:38:23
2024-01-08 06:13:23
2024-01-08 02:40:35
2024-01-07 22:50:23
2024-01-07 16:48:59
2024-01-07 09:09:59
2024-01-07 01:22:35
2024-01-06 19:05:47
2024-01-06 12:44:11
2024-01-06 07:03:35
2024-01-06 02:24:59
2024-01-05 22:07:47
2024-01-05 18:10:11
2024-01-05 14:44:11
2024-01-05 10:58:23
2024-01-05 05:55:47
2024-01-05 01:59:11
2024-01-04 22:23:47
2024-01-04 19:13:11
2024-01-04 15:37:35
2024-01-04 11:51:35
2024-01-04 07:26:59
2024-01-04 03:53:23
2024-01-03 23:55:59
2024-01-03 19:24:47
2024-01-03 16:17:35
2024-01-03 13:45:59
2024-01-03 11:58:23
2024-01-03 07:28:11
2024-01-03 02:37:59
2024-01-02 22:41:47
2024-01-02 18:54:47
2024-01-02 15:16:11
